In [2]:
import numpy as np
import os
import cv2
from tensorflow.keras import applications, models, layers

In [6]:
# 데이터 전처리
def get_data(path):
    x = []
    y = []
    dir_list = os.listdir(path)
    for i in range(len(dir_list)-1):
        dir_path = path + "/" + dir_list[i+1]
        dir_name = os.listdir(dir_path)

        for j in range(len(dir_name)):
            full_dir_path = dir_path + "/" + dir_name[j]

            img = cv2.imread(full_dir_path)
            img = cv2.resize(img, (32,32))
            x.append(img)
            y.append(i)
    # print(x)
    # print(y)
    return np.array(x),np.array(y)

# 모델 생성
def make_model() :
    resnet50 = applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3))
    resnet50.trainable = False
    model = models.Sequential()
    model.add(resnet50)
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(512, activation="relu"))

    # 분류 데이터가 몇종류인지
    model.add(layers.Dense(2, activation="softmax"))
    model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
    print(model.summary())
    return model

# 모델 학습
def learning() :
    train_path = "./data_image"
    test_path = "./test"
    (x_train, y_train)= get_data(train_path)
    (x_test, y_test) = get_data(test_path)
    model = make_model()
    # print("x_train : ",x_train[0])
    # print("x_train.shape : ",x_train[0].shape)
    log = model.fit(x_train, y_train, epochs=2, batch_size=16)
    #model.save("face.h5")
    
    model.evaluate(x_test,y_test)

In [7]:
train_path = "./data_image"
test_path = "./test"
(x_train, y_train)= get_data(train_path)
(x_test, y_test) = get_data(test_path)
model = make_model()
# print("x_train : ",x_train[0])
# print("x_train.shape : ",x_train[0].shape)
log = model.fit(x_train, y_train, epochs=2, batch_size=16)
#model.save("face.h5")

model.evaluate(x_test,y_test)

In [ ]:
import matplotlib.pyplot as plt
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx() 
loss_ax.plot(log.history['loss'], 'y', label='train loss') 
loss_ax.plot(log.history['val_loss'], 'r', label='val loss') 
acc_ax.plot(log.history['accuracy'], 'b', label='train acc')
acc_ax.plot(log.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch') 
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='lower right')
acc_ax.legend(loc='upper right')
plt.show()